In [ ]:
#参数解读https://blog.csdn.net/TeFuirnever/article/details/99818078

### 导入数据

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel('data_test_train.xlsx')
data.head()

,comment,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,0
1,菜品丰富质量好，服务也不错！很喜欢！,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,1
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,1


### KNN算法

#### 数据预处理 

In [3]:
#根据任务需要做处理
#数据去重、删除空值

#### jieba分词

In [4]:
import jieba
#根据需要补充完善分词功能，可参照词频分析或者LDA主题模型的代码
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

data['cut_comment'] = data.comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\17888\AppData\Local\Temp\jieba.cache
Loading model cost 1.449 seconds.
Prefix dict has been built successfully.


In [5]:
data.head()

,comment,sentiment,cut_comment
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,0,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 ? ? 我 感觉 口...
1,菜品丰富质量好，服务也不错！很喜欢！,1,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,1,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,1,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 ? ? 每次 出去 回来 总...


#### 提取特征

In [6]:
from sklearn.feature_extraction.text import CountVectorizer  #TfidfVectorizer

def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = '哈工大停用词表.txt'
stopwords = get_custom_stopwords(stop_words_file)

vect = CountVectorizer(max_df = 0.8, 
                       min_df = 3, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

#### 划分数据集

In [7]:
#划分数据集
X = data['cut_comment']
y = data.sentiment

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [8]:
#特征展示
test = pd.DataFrame(vect.fit_transform(X_train).toarray(), columns=vect.get_feature_names())
test.head()

D:\software\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,ipad,ok,一下,一个个,一个半,一个多,一人,一份,一会,一会儿,...,麻烦,麻辣,麻酱,黄瓜,黄盖,黄色,黏糊糊,黑椒,默默,齐全
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 训练模型

In [32]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #,weights='distance'

X_train_vect = vect.fit_transform(X_train)
knn.fit(X_train_vect, y_train)
train_score = knn.score(X_train_vect, y_train)
print(train_score)  # train_score 模型在训练数据上的效果

1.0


#### 测试模型

In [33]:
X_test_vect = vect.transform(X_test)
print(knn.score(X_test_vect, y_test)) # 模型在测试数据上的效果

0.6984126984126984


#### 分析数据 

In [11]:
data = pd.read_excel("data.xlsx")
data.head()

,comment
0,菜品口味还行，上菜太慢，一杯鲜榨果汁居然四十分钟才上。薯条非常好吃，是我吃过最好吃的
1,铁板蛏子都是沙子，酱爆海兔超咸，海参捞饭里的海参硬硬的。评论都是骗人的……
2,重盐??油大??号称东北菜??东北菜什么时候这么油了？唯一的一个素菜??那个油啊??唉茄盒创...
3,有点流于表面??菜相当一般??甚至有点难吃??别的都OK??但是饭馆最重要的还是菜啊??很坑
4,"太差了,见过次的没见过这么次的。首先说稍显不次的，平心而论他的口味实在是太对不起它的价格了价..."


In [12]:
data = pd.read_excel("data.xlsx")
data['cut_comment'] = data.comment.apply(chinese_word_cut)
X=data['cut_comment']
X_vec = vect.transform(X)
knn_result = knn.predict(X_vec)
#predict_proba(X)[source] 返回概率
data['knn_sentiment'] = knn_result

In [13]:
data.to_excel("data_result.xlsx",index=False)